In [2]:
import googlemaps
import pandas as pd
import pickle

In [2]:
# keyfile = '/dartfs/rc/lab/P/Psych81.09/course/shared/gmaps.api.key'
keyfile = 'gmaps.api.key'
fd = open(keyfile, 'r')
key = fd.readline()[:-1]
fd.close()

In [3]:
gmaps = googlemaps.Client(key=key)

In [8]:
df=pd.read_csv('/Users/paxtonfitzpatrick/Desktop/parking-violations-geodata.csv',
              delimiter=',', encoding = 'latin-1', index_col='Unnamed: 0', dtype={'Zip Code' : int})

In [5]:
geodata = dict()

for ix, row in df.iterrows():
    fmt_addr = str(row['House Number']) + ' ' + row['Street Name'] + ', ' + row['City & State']
    try:
#         addr_info = gmaps.geocode(fmt_addr)
    except:
        print(f'Error in row {ix}')
        continue
        
    if addr_info:
        geodata[ix] = addr_info
    else:
        print(f'No data returned for row {ix}')
        

In [8]:
# with open('geodata.p', 'wb') as f:
#     pickle.dump(geodata, f)

In [50]:
for ix, row in df.iterrows():
    for component in geodata[ix][0]['address_components']:
        if 'postal_code' in component['types']:   
            df.at[ix, 'Zip Code'] = int(component['short_name'])
            
    df.at[ix, 'Lat'] = geodata[ix][0]['geometry']['location']['lat']
    df.at[ix, 'Long'] = geodata[ix][0]['geometry']['location']['lng']

In [56]:
# df.to_csv('parking-violations-geodata.csv')

In [3]:
with open('geodata.p', 'rb') as f:
    geodata = pickle.load(f)

In [11]:
geodata[4]

[{'address_components': [{'long_name': '133',
    'short_name': '133',
    'types': ['street_number']},
   {'long_name': 'Frost Street', 'short_name': 'Frost St', 'types': ['route']},
   {'long_name': 'Williamsburg',
    'short_name': 'Williamsburg',
    'types': ['neighborhood', 'political']},
   {'long_name': 'Brooklyn',
    'short_name': 'Brooklyn',
    'types': ['political', 'sublocality', 'sublocality_level_1']},
   {'long_name': 'Kings County',
    'short_name': 'Kings County',
    'types': ['administrative_area_level_2', 'political']},
   {'long_name': 'New York',
    'short_name': 'NY',
    'types': ['administrative_area_level_1', 'political']},
   {'long_name': 'United States',
    'short_name': 'US',
    'types': ['country', 'political']},
   {'long_name': '11211', 'short_name': '11211', 'types': ['postal_code']}],
  'formatted_address': '133 Frost St, Brooklyn, NY 11211, USA',
  'geometry': {'bounds': {'northeast': {'lat': 40.7181074, 'lng': -73.9462073},
    'southwest': {'

In [9]:
df.head(5)

,Summons Number,Violation Code,Violation Time,House Number,Street Name,City & State,Zip Code,Lat,Long
0,8725653684,71,0319P,235,Bedford Ave,"Brooklyn, NY",11211,40.715957,-73.959527
1,8725653696,20,0322P,235,Bedford Ave,"Brooklyn, NY",11211,40.715957,-73.959527
2,8725653702,20,0324P,235,Bedford Ave,"Brooklyn, NY",11211,40.715957,-73.959527
3,8670481194,21,1138A,133,Frost St,"Brooklyn, NY",11211,40.718053,-73.946250
4,8670481212,21,1140A,133,Frost St,"Brooklyn, NY",11211,40.718053,-73.946250


In [10]:
df['Zip Code']

0      11211
1      11211
2      11211
3      11211
4      11211
5      11211
6      11211
7      11211
8      11211
9      11211
10     11211
11     11211
12     11211
13     11211
14     11211
15     11222
16     11211
17     11211
18     11215
19     11211
20     11211
21     11211
22     11211
23     11211
24     11211
25     11211
26     11211
27     11211
28     11211
29     11211
       ...  
343    11211
344    11211
345    11211
346    11233
347    11233
348    11222
349    11222
350    11222
351    11222
352    11222
353    11222
354    11222
355    11211
356    11211
357    11211
358    11211
359    11211
360    11211
361    11211
362    11211
363    11211
364    11211
365    11249
366    11249
367    11249
368    11249
369    11249
370    11249
371    11249
372    11249
Name: Zip Code, Length: 373, dtype: int64